Приклад рекомендаційної системи на основі контента
===
Імпортуємо необхідні бібліотеки
---

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Зчитуємо дані - книги
--

In [2]:
df = pd.read_csv('datasets/book-crossing/Books.csv',
                on_bad_lines='skip',
                 sep=';'
                )

In [3]:
df.head()

,ISBN,Title,Author,Year,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company


Завантажуємо дані з користувачами та рейтингами 


In [4]:
user_ratings = pd.read_csv('datasets/book-crossing/users-ratings.csv')

Відфільтруємо дані - працюємо тільки з підмножиною книг, які були прочитані хочаб одним користувачем

In [5]:
df = df[df['ISBN'].isin(user_ratings['ISBN'])]

* Заповнимо порожні значення
* Зтворимо нову змінну text, яка є поєднанням автору та назви

In [6]:
df.dropna(subset=['Title','Author'],inplace=True)
df.drop_duplicates(subset=['Title','Author'], inplace=True)
df['Title'] = df['Title'].apply(lambda s: s.lower())
df['Author'] = df['Author'].apply(lambda s: s.lower())
df['text'] = df['Title'].fillna('') + ' ' + df['Author'].fillna('')

* Заберемо дублікати за автором та назвою
* Приберемо порожні значення

Оскільки наступний код спирається на індекси книг у датафрейми, зробимо reset. 

In [8]:
df.reset_index(drop=True, inplace=True)

Застосовуємо tfidf для трансформації текста в матрицю

In [9]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['text'])

Рахуємо косинуси між векторами

In [10]:
cosine_sim = cosine_similarity(tfidf_matrix)

In [11]:
cosine_sim.shape

(1772, 1772)

Знайдемо подібні книги до заданої.

In [19]:
def get_recommendations(isbn, cosine_sim=cosine_sim, top_n = 10):

    idx = df[df['ISBN']==isbn].index

    sim_scores = cosine_sim[idx][0]

    sim_scores = sorted(list(enumerate(sim_scores)), key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:top_n + 1] 

    book_indices = [i[0] for i in sim_scores]

    return df.iloc[book_indices]

In [20]:
df[df['ISBN']=='0440234743']

,ISBN,Title,Author,Year,Publisher,text
0,0440234743,the testament,john grisham,1999,Dell,the testament john grisham


In [21]:
get_recommendations('0440234743')

,ISBN,Title,Author,Year,Publisher,text
153,0385424728,the chamber,john grisham,1994,Doubleday Books,the chamber john grisham
154,0385472951,the partner,john grisham,1997,Doubleday Books,the partner john grisham
322,0385510438,the last juror,john grisham,2004,Doubleday,the last juror john grisham
61,0385497466,the brethren,john grisham,2000,Doubleday,the brethren john grisham
64,0385511612,bleachers,john grisham,2003,Doubleday,bleachers john grisham
99,038542471X,the client,john grisham,1993,Doubleday Books,the client john grisham
114,044021145X,the firm,john grisham,1992,Bantam Dell Publishing Group,the firm john grisham
228,044022165X,the rainmaker,john grisham,1996,Dell,the rainmaker john grisham
854,0440241073,the summons,john grisham,2002,Dell Publishing Company,the summons john grisham
62,0385508042,the king of torts,john grisham,2003,Doubleday Books,the king of torts john grisham


Перепишемо фукцію, де тепер рекомендації будуть підбиратися для користувача. Враховуючу його уподобання, та прочитані книги.

In [ ]:
(user_ratings[(user_ratings['User-ID']==243)&(user_ratings['Rating']>8)]
.join(df.set_index('ISBN'), on='ISBN'))